# atmodeller

## Tutorial 1: basic operation

Import the required packages and set the package logger to the INFO level. For more output you could instead set it to DEBUG.

In [1]:
from atmodeller import InteriorAtmosphereSystem, Molecule, OCEAN_MOLES, MolarMasses, logger, FugacityConstraint, BufferedFugacityConstraint, SystemConstraint, MassConstraint
from atmodeller.thermodynamics import PeridotiteH2O, NoSolubility, BasaltDixonCO2, StandardGibbsFreeEnergyOfFormationHolland, StandardGibbsFreeEnergyOfFormationLinear

import logging

logger.setLevel(logging.DEBUG)

09:43:26 - atmodeller                     - INFO      - atmodeller version 0.1.0


### 1. Simple H2O-H2 system with prescribed H2O pressure

We define a list of the molecules we wish to include in the interior-atmosphere system and how they partition between the melt and the atmosphere, and the solid and the melt:

In [2]:
molecules: list[Molecule] = []
molecules.append(Molecule(name='H2O', solubility=PeridotiteH2O(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='H2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))

09:43:26 - atmodeller.core                - INFO      - Creating a molecule: H2O
09:43:26 - atmodeller.core                - DEBUG     - element count = 
{'H': 2, 'O': 1}
09:43:26 - atmodeller.core                - INFO      - Creating a molecule: H2
09:43:26 - atmodeller.core                - DEBUG     - element count = 
{'H': 2}


Although a choice is often made to constrain oxygen fugacity, we nevertheless need to explicitly include O2 as a molecule in the interior-atmosphere system:

In [3]:
molecules.append(Molecule(name='O2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules

09:43:26 - atmodeller.core                - INFO      - Creating a molecule: O2
09:43:26 - atmodeller.core                - DEBUG     - element count = 
{'O': 2}


[Molecule(name='H2O', solubility=<atmodeller.thermodynamics.PeridotiteH2O object at 0x123e3b5d0>, solid_melt_distribution_coefficient=0, elements={'H': 2, 'O': 1}, element_masses={'H': 0.0020158, 'O': 0.0159994}, molar_mass=0.018015200000000002, output=None),
 Molecule(name='H2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x10cf76210>, solid_melt_distribution_coefficient=0, elements={'H': 2}, element_masses={'H': 0.0020158}, molar_mass=0.0020158, output=None),
 Molecule(name='O2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x12433df90>, solid_melt_distribution_coefficient=0, elements={'O': 2}, element_masses={'O': 0.0319988}, molar_mass=0.0319988, output=None)]

We can then create an interior-atmosphere system using the list of molecules and (optionally) specifying the thermodynamic data to use. Note that this creates a planet with 'default properties' (a molten Earth). Adjusting the planet properties will be covered in a later tutorial.

In [4]:
interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(molecules=molecules, gibbs_data=StandardGibbsFreeEnergyOfFormationLinear())

09:43:26 - atmodeller.core                - INFO      - Creating a new planet
09:43:26 - atmodeller.core                - INFO      - Mantle mass (kg) = 4.208261222595111e+24
09:43:26 - atmodeller.core                - INFO      - Mantle melt fraction = 1.0
09:43:26 - atmodeller.core                - INFO      - Core mass fraction = 0.295334691460966
09:43:26 - atmodeller.core                - INFO      - Planetary radius (m) = 6371000.0
09:43:26 - atmodeller.core                - INFO      - Planetary mass (kg) = 5.972e+24
09:43:26 - atmodeller.core                - INFO      - Surface temperature (K) = 2000.000000
09:43:26 - atmodeller.core                - INFO      - Surface gravity (m/s^2) = 9.819973426224687
09:43:26 - atmodeller.core                - INFO      - Melt Composition = None
09:43:26 - atmodeller.core                - INFO      - Creating a new interior-atmosphere system
09:43:26 - atmodeller.core                - INFO      - Molecules = ['H2', 'O2', 'H2O']
09:43:26 -

Notice that for this simple system it has identified the single reaction that relates the 3 chosen molecules. To solve the system, we provide a constraint of the H2O pressure in bar:

In [5]:
H2O_pressure: SystemConstraint = FugacityConstraint(species='H2O', value=1)
constraints: list = [H2O_pressure]

A second constraint is required to close the system of equations, and often this is constraining the fugacity of some species at a buffered value, such as constraining the oxygen fugacity at the iron-wustite (IW) buffer. Constraining the fO2 at the IW buffer is the default if no arguments are provided:

In [6]:
O2_pressure: SystemConstraint = BufferedFugacityConstraint()
constraints.append(O2_pressure)
logger.info(constraints)

09:43:26 - atmodeller                     - INFO      - [FugacityConstraint(species='H2O', value=1, field='fugacity'), BufferedFugacityConstraint(species='O2', fugacity=<atmodeller.thermodynamics.IronWustiteBufferOneill object at 0x124355ed0>, log10_shift=0, field='fugacity')]


In [7]:
interior_atmosphere.solve(constraints)

09:43:26 - atmodeller.core                - INFO      - Constraints: [FugacityConstraint(species='H2O', value=1, field='fugacity'),
 BufferedFugacityConstraint(species='O2',
                            fugacity=<atmodeller.thermodynamics.IronWustiteBufferOneill object at 0x124355ed0>,
                            log10_shift=0,
                            field='fugacity')]
09:43:26 - atmodeller.core                - INFO      - Mixed pressure and mass constraints so attempting to solve a non-linear system of equations
09:43:26 - atmodeller.core                - INFO      - The necessary number of constraints will be applied to the reaction network to solve the system
09:43:26 - atmodeller.core                - INFO      - Row 00: Reaction 0: 1.0 H2 + 0.5 O2 = 1.0 H2O
09:43:26 - atmodeller.thermodynamics      - DEBUG     - Molecule = H2, standard Gibbs energy of formation = 0.000000
09:43:26 - atmodeller.thermodynamics      - DEBUG     - Molecule = O2, standard Gibbs energy of formation

{'H2': 1.4662542407051478, 'O2': 3.910416694617766e-08, 'H2O': 1.0}

You can access the solution directly using:

In [8]:
interior_atmosphere.fugacities_dict

{'H2': 1.4662542407051478, 'O2': 3.910416694617766e-08, 'H2O': 1.0}

You can access more information about the inputs and outputs using:

In [9]:
interior_atmosphere

InteriorAtmosphereSystem(molecules=[Molecule(name='H2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x10cf76210>, solid_melt_distribution_coefficient=0, elements={'H': 2}, element_masses={'H': 0.0020158}, molar_mass=0.0020158, output=None), Molecule(name='O2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x12433df90>, solid_melt_distribution_coefficient=0, elements={'O': 2}, element_masses={'O': 0.0319988}, molar_mass=0.0319988, output=None), Molecule(name='H2O', solubility=<atmodeller.thermodynamics.PeridotiteH2O object at 0x123e3b5d0>, solid_melt_distribution_coefficient=0, elements={'H': 2, 'O': 1}, element_masses={'H': 0.0020158, 'O': 0.0159994}, molar_mass=0.018015200000000002, output=None)], gibbs_data=<atmodeller.thermodynamics.StandardGibbsFreeEnergyOfFormationLinear object at 0x1243235d0>, planet=Planet(mantle_mass=4.208261222595111e+24, mantle_melt_fraction=1.0, core_mass_fraction=0.295334691460966, surface_radius=6371000.0, surface_temperatur

### 2. System with C and H and prescribed pressures

We now extend the molecule list to additionally include C-species:

In [10]:
molecules: list[Molecule] = []
molecules.append(Molecule(name='H2O', solubility=PeridotiteH2O(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='H2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='O2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='CO', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='CO2', solubility=BasaltDixonCO2(), solid_melt_distribution_coefficient=0))
molecules

09:43:26 - atmodeller.core                - INFO      - Creating a molecule: H2O
09:43:26 - atmodeller.core                - DEBUG     - element count = 
{'H': 2, 'O': 1}
09:43:26 - atmodeller.core                - INFO      - Creating a molecule: H2
09:43:26 - atmodeller.core                - DEBUG     - element count = 
{'H': 2}
09:43:26 - atmodeller.core                - INFO      - Creating a molecule: O2
09:43:26 - atmodeller.core                - DEBUG     - element count = 
{'O': 2}
09:43:26 - atmodeller.core                - INFO      - Creating a molecule: CO
09:43:26 - atmodeller.core                - DEBUG     - element count = 
{'C': 1, 'O': 1}
09:43:26 - atmodeller.core                - INFO      - Creating a molecule: CO2
09:43:26 - atmodeller.core                - DEBUG     - element count = 
{'C': 1, 'O': 2}


[Molecule(name='H2O', solubility=<atmodeller.thermodynamics.PeridotiteH2O object at 0x12435d5d0>, solid_melt_distribution_coefficient=0, elements={'H': 2, 'O': 1}, element_masses={'H': 0.0020158, 'O': 0.0159994}, molar_mass=0.018015200000000002, output=None),
 Molecule(name='H2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x11b804e10>, solid_melt_distribution_coefficient=0, elements={'H': 2}, element_masses={'H': 0.0020158}, molar_mass=0.0020158, output=None),
 Molecule(name='O2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x12433e550>, solid_melt_distribution_coefficient=0, elements={'O': 2}, element_masses={'O': 0.0319988}, molar_mass=0.0319988, output=None),
 Molecule(name='CO', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x11a52eed0>, solid_melt_distribution_coefficient=0, elements={'C': 1, 'O': 1}, element_masses={'C': 0.0120107, 'O': 0.0159994}, molar_mass=0.0280101, output=None),
 Molecule(name='CO2', solubility=<atmodeller.th

In [11]:
interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(molecules=molecules)

09:43:26 - atmodeller.core                - INFO      - Creating a new planet
09:43:26 - atmodeller.core                - INFO      - Mantle mass (kg) = 4.208261222595111e+24
09:43:27 - atmodeller.core                - INFO      - Mantle melt fraction = 1.0
09:43:27 - atmodeller.core                - INFO      - Core mass fraction = 0.295334691460966
09:43:27 - atmodeller.core                - INFO      - Planetary radius (m) = 6371000.0
09:43:27 - atmodeller.core                - INFO      - Planetary mass (kg) = 5.972e+24
09:43:27 - atmodeller.core                - INFO      - Surface temperature (K) = 2000.000000
09:43:27 - atmodeller.core                - INFO      - Surface gravity (m/s^2) = 9.819973426224687
09:43:27 - atmodeller.core                - INFO      - Melt Composition = None
09:43:27 - atmodeller.core                - INFO      - Creating a new interior-atmosphere system
09:43:27 - atmodeller.core                - INFO      - Molecules = ['CO', 'H2', 'O2', 'CO2', 'H2O

Note now the system has identified two reactions in the network. With C present in the system we must provide at least 2 constraints, in addition to the oxygen fugacity:

In [12]:
H2O_pressure: SystemConstraint = FugacityConstraint(species='H2O', value=1)
CO2_pressure: SystemConstraint = FugacityConstraint(species='CO2', value=1)
O2_pressure: SystemConstraint = BufferedFugacityConstraint()
constraints: list[SystemConstraint] = [H2O_pressure, CO2_pressure, O2_pressure]

In [13]:
interior_atmosphere.solve(constraints)
interior_atmosphere.fugacities_dict

09:43:27 - atmodeller.core                - INFO      - Constraints: [FugacityConstraint(species='H2O', value=1, field='fugacity'),
 FugacityConstraint(species='CO2', value=1, field='fugacity'),
 BufferedFugacityConstraint(species='O2',
                            fugacity=<atmodeller.thermodynamics.IronWustiteBufferOneill object at 0x124356050>,
                            log10_shift=0,
                            field='fugacity')]
09:43:27 - atmodeller.core                - INFO      - Mixed pressure and mass constraints so attempting to solve a non-linear system of equations
09:43:27 - atmodeller.core                - INFO      - The necessary number of constraints will be applied to the reaction network to solve the system
09:43:27 - atmodeller.core                - INFO      - Row 00: Reaction 0: 1.0 CO + 0.5 O2 = 1.0 CO2
09:43:27 - atmodeller.thermodynamics      - DEBUG     - Molecule = CO, standard Gibbs energy of formation = -285748.794118
09:43:27 - atmodeller.thermodynamics

{'CO': 6.580633105623207,
 'H2': 1.4662542407051478,
 'O2': 3.910416694617766e-08,
 'CO2': 1.0,
 'H2O': 1.0}

There is not a requirement to necessarily impose the oxygen fugacity as a constraint. Instead, we can simply impose three pressure constraints (that span the reaction set) and allow for the oxygen fugacity to be solved. Note that if we do not specify an appropriate range of constraints we cannot solve the system of equations to give a unique solution and hence the code will raise an exception relating to a singular matrix.

In [14]:
H2O_pressure: SystemConstraint = FugacityConstraint(species='H2O', value=1)
H2_pressure: SystemConstraint = FugacityConstraint(species='H2', value=1)
CO_pressure: SystemConstraint = FugacityConstraint(species='CO', value=1)
constraints: list[SystemConstraint] = [H2O_pressure, H2_pressure, CO_pressure]

In [15]:
interior_atmosphere.solve(constraints)
interior_atmosphere.fugacities_dict

09:43:27 - atmodeller.core                - INFO      - Constraints: [FugacityConstraint(species='H2O', value=1, field='fugacity'),
 FugacityConstraint(species='H2', value=1, field='fugacity'),
 FugacityConstraint(species='CO', value=1, field='fugacity')]
09:43:27 - atmodeller.core                - INFO      - Mixed pressure and mass constraints so attempting to solve a non-linear system of equations
09:43:27 - atmodeller.core                - INFO      - The necessary number of constraints will be applied to the reaction network to solve the system
09:43:27 - atmodeller.core                - INFO      - Row 00: Reaction 0: 1.0 CO + 0.5 O2 = 1.0 CO2
09:43:27 - atmodeller.thermodynamics      - DEBUG     - Molecule = CO, standard Gibbs energy of formation = -285748.794118
09:43:27 - atmodeller.thermodynamics      - DEBUG     - Molecule = H2, standard Gibbs energy of formation = 0.000000
09:43:27 - atmodeller.thermodynamics      - DEBUG     - Molecule = O2, standard Gibbs energy of format

{'CO': 1.0,
 'H2': 1.0,
 'O2': 8.407010711071694e-08,
 'CO2': 0.22281355261277522,
 'H2O': 1.0}

### 3. System with C and H and mixed constraints

A typical use case is to define an interior-atmosphere system with a combination of pressure and mass constraints. We define the same molecule set as before:

In [16]:
molecules: list[Molecule] = []
molecules.append(Molecule(name='H2O', solubility=PeridotiteH2O(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='H2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='O2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='CO', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='CO2', solubility=BasaltDixonCO2(), solid_melt_distribution_coefficient=0))
molecules

09:43:27 - atmodeller.core                - INFO      - Creating a molecule: H2O
09:43:27 - atmodeller.core                - DEBUG     - element count = 
{'H': 2, 'O': 1}
09:43:27 - atmodeller.core                - INFO      - Creating a molecule: H2
09:43:27 - atmodeller.core                - DEBUG     - element count = 
{'H': 2}
09:43:27 - atmodeller.core                - INFO      - Creating a molecule: O2
09:43:27 - atmodeller.core                - DEBUG     - element count = 
{'O': 2}
09:43:27 - atmodeller.core                - INFO      - Creating a molecule: CO
09:43:27 - atmodeller.core                - DEBUG     - element count = 
{'C': 1, 'O': 1}
09:43:27 - atmodeller.core                - INFO      - Creating a molecule: CO2
09:43:27 - atmodeller.core                - DEBUG     - element count = 
{'C': 1, 'O': 2}


[Molecule(name='H2O', solubility=<atmodeller.thermodynamics.PeridotiteH2O object at 0x12436ec90>, solid_melt_distribution_coefficient=0, elements={'H': 2, 'O': 1}, element_masses={'H': 0.0020158, 'O': 0.0159994}, molar_mass=0.018015200000000002, output=None),
 Molecule(name='H2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x12435c250>, solid_melt_distribution_coefficient=0, elements={'H': 2}, element_masses={'H': 0.0020158}, molar_mass=0.0020158, output=None),
 Molecule(name='O2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x104e04950>, solid_melt_distribution_coefficient=0, elements={'O': 2}, element_masses={'O': 0.0319988}, molar_mass=0.0319988, output=None),
 Molecule(name='CO', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x12435a5d0>, solid_melt_distribution_coefficient=0, elements={'C': 1, 'O': 1}, element_masses={'C': 0.0120107, 'O': 0.0159994}, molar_mass=0.0280101, output=None),
 Molecule(name='CO2', solubility=<atmodeller.th

Now we define the constraints, and in this case we want to constrain the total mass of C and H in the system that can partition between the various reservoirs.

In [17]:
number_of_earth_oceans: float = 1
# C/H ratio by mass.
ch_ratio: float = 1

mass_H: float = number_of_earth_oceans * OCEAN_MOLES * MolarMasses().H2
mass_C: float = ch_ratio * mass_H

constraints: list[SystemConstraint] = [
    MassConstraint(species="H", value=mass_H),
    MassConstraint(species="C", value=mass_C),
    BufferedFugacityConstraint()
]

interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(molecules=molecules)
interior_atmosphere.solve(constraints)
interior_atmosphere.fugacities_dict

09:43:27 - atmodeller.core                - INFO      - Creating a new planet
09:43:27 - atmodeller.core                - INFO      - Mantle mass (kg) = 4.208261222595111e+24
09:43:27 - atmodeller.core                - INFO      - Mantle melt fraction = 1.0
09:43:27 - atmodeller.core                - INFO      - Core mass fraction = 0.295334691460966
09:43:27 - atmodeller.core                - INFO      - Planetary radius (m) = 6371000.0
09:43:27 - atmodeller.core                - INFO      - Planetary mass (kg) = 5.972e+24
09:43:27 - atmodeller.core                - INFO      - Surface temperature (K) = 2000.000000
09:43:27 - atmodeller.core                - INFO      - Surface gravity (m/s^2) = 9.819973426224687
09:43:27 - atmodeller.core                - INFO      - Melt Composition = None
09:43:27 - atmodeller.core                - INFO      - Creating a new interior-atmosphere system
09:43:27 - atmodeller.core                - INFO      - Molecules = ['CO', 'H2', 'O2', 'CO2', 'H2O

{'CO': 62.28167551155328,
 'H2': 0.5760309184494737,
 'O2': 3.910416694617766e-08,
 'CO2': 9.464389597762718,
 'H2O': 0.3928588252010446}

### 4. Including more species such as CH4

It is straightforward to add more species to the system, although they must have their formation energies and masses already specified in the code.

In [18]:
molecules: list[Molecule] = []
molecules.append(Molecule(name='H2O', solubility=PeridotiteH2O(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='H2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='O2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='CO', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='CO2', solubility=BasaltDixonCO2(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='CH4', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules

09:43:27 - atmodeller.core                - INFO      - Creating a molecule: H2O
09:43:27 - atmodeller.core                - DEBUG     - element count = 
{'H': 2, 'O': 1}
09:43:27 - atmodeller.core                - INFO      - Creating a molecule: H2
09:43:27 - atmodeller.core                - DEBUG     - element count = 
{'H': 2}
09:43:27 - atmodeller.core                - INFO      - Creating a molecule: O2
09:43:27 - atmodeller.core                - DEBUG     - element count = 
{'O': 2}
09:43:27 - atmodeller.core                - INFO      - Creating a molecule: CO
09:43:27 - atmodeller.core                - DEBUG     - element count = 
{'C': 1, 'O': 1}
09:43:27 - atmodeller.core                - INFO      - Creating a molecule: CO2
09:43:27 - atmodeller.core                - DEBUG     - element count = 
{'C': 1, 'O': 2}
09:43:27 - atmodeller.core                - INFO      - Creating a molecule: CH4
09:43:27 - atmodeller.core                - DEBUG     - element count = 
{'C': 1, '

[Molecule(name='H2O', solubility=<atmodeller.thermodynamics.PeridotiteH2O object at 0x124379d10>, solid_melt_distribution_coefficient=0, elements={'H': 2, 'O': 1}, element_masses={'H': 0.0020158, 'O': 0.0159994}, molar_mass=0.018015200000000002, output=None),
 Molecule(name='H2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x11a5228d0>, solid_melt_distribution_coefficient=0, elements={'H': 2}, element_masses={'H': 0.0020158}, molar_mass=0.0020158, output=None),
 Molecule(name='O2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x12435fed0>, solid_melt_distribution_coefficient=0, elements={'O': 2}, element_masses={'O': 0.0319988}, molar_mass=0.0319988, output=None),
 Molecule(name='CO', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x11b630710>, solid_melt_distribution_coefficient=0, elements={'C': 1, 'O': 1}, element_masses={'C': 0.0120107, 'O': 0.0159994}, molar_mass=0.0280101, output=None),
 Molecule(name='CO2', solubility=<atmodeller.th

We define a mixture of mass and oxygen fugacity constraints as before and solve the system. CH4 is not prevalent at 2000 K so the results are almost identical to those without CH4 presented above.

In [19]:
number_of_earth_oceans: float = 1
# C/H ratio by mass.
ch_ratio: float = 1

mass_H: float = number_of_earth_oceans * OCEAN_MOLES * MolarMasses().H2
mass_C: float = ch_ratio * mass_H

constraints: list[SystemConstraint] = [
    MassConstraint(species="H", value=mass_H),
    MassConstraint(species="C", value=mass_C),
    BufferedFugacityConstraint()
]

interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(molecules=molecules)
interior_atmosphere.solve(constraints)
interior_atmosphere.fugacities_dict

09:43:27 - atmodeller.core                - INFO      - Creating a new planet
09:43:27 - atmodeller.core                - INFO      - Mantle mass (kg) = 4.208261222595111e+24
09:43:27 - atmodeller.core                - INFO      - Mantle melt fraction = 1.0
09:43:27 - atmodeller.core                - INFO      - Core mass fraction = 0.295334691460966
09:43:27 - atmodeller.core                - INFO      - Planetary radius (m) = 6371000.0
09:43:27 - atmodeller.core                - INFO      - Planetary mass (kg) = 5.972e+24
09:43:27 - atmodeller.core                - INFO      - Surface temperature (K) = 2000.000000
09:43:27 - atmodeller.core                - INFO      - Surface gravity (m/s^2) = 9.819973426224687
09:43:27 - atmodeller.core                - INFO      - Melt Composition = None
09:43:27 - atmodeller.core                - INFO      - Creating a new interior-atmosphere system
09:43:27 - atmodeller.core                - INFO      - Molecules = ['CO', 'H2', 'O2', 'CO2', 'H2O

{'CO': 62.28167381126519,
 'H2': 0.5760309112266976,
 'O2': 3.910416694617766e-08,
 'CO2': 9.464389339385132,
 'H2O': 0.39285882027503943,
 'CH4': 1.3829358485053396e-06}